# Att skicka pandas DataFrame genom pipelines istf arrays
## importeras från sklearn_pandas

In [5]:
!pip install sklearn_pandas

import pandas as pd
import numpy as np
from sklearn_pandas import DataFrameMapper, gen_features
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

## Read data
We are not reading the original data from Kaggel.  
The data we are reading is based on the data from Kaggle messed up with small changes:
* Random replacement of values with NaNs
* Transformations of categorical (int) features to text categories

In [6]:
np.random.seed(seed=42)
df_data = pd.read_csv("./cardiovascular-disease-dataset/messy/cardio_train.csv", sep=';', index_col="id")
display(df_data.describe(include="all"))
display(df_data.head())

age gender        height        weight         ap_hi  \
count   70000.000000  70000  66591.000000  66578.000000  70000.000000   
unique           NaN      2           NaN           NaN           NaN   
top              NaN  women           NaN           NaN           NaN   
freq             NaN  45530           NaN           NaN           NaN   
mean    19468.865814    NaN    164.361205     74.210467    128.817286   
std      2467.251667    NaN      8.226411     14.397678    154.011419   
min     10798.000000    NaN     55.000000     10.000000   -150.000000   
25%     17664.000000    NaN    159.000000     65.000000    120.000000   
50%     19703.000000    NaN    165.000000     72.000000    120.000000   
75%     21327.000000    NaN    170.000000     82.000000    140.000000   
max     23713.000000    NaN    250.000000    200.000000  16020.000000   

               ap_lo cholesterol    gluc         smoke          alco  \
count   70000.000000       66589   70000  70000.000000  70000.000000   
unique           NaN           3       3           NaN           NaN   
top              NaN      normal  normal           NaN           NaN   
freq             NaN       49789   59479           NaN           NaN   
mean       96.630414         NaN     NaN      0.088129      0.053771   
std       188.472530         NaN     NaN      0.283484      0.225568   
min       -70.000000         NaN     NaN      0.000000      0.000000   
25%        80.000000         NaN     NaN      0.000000      0.000000   
50%        80.000000         NaN     NaN      0.000000      0.000000   
75%        90.000000         NaN     NaN      0.000000      0.000000   
max     11000.000000         NaN     NaN      1.000000      1.000000   

              active        cardio  
count   70000.000000  70000.000000  
unique           NaN           NaN  
top              NaN           NaN  
freq             NaN           NaN  
mean        0.803729      0.499700  
std         0.397179      0.500003  
min         0.000000      0.000000  
25%         1.000000      0.000000  
50%         1.000000      0.000000  
75%         1.000000      1.000000  
max         1.000000      1.000000

age gender  height  weight  ap_hi  ap_lo        cholesterol    gluc  \
id                                                                          
0   18393    men   168.0    62.0    110     80             normal  normal   
1   20228  women   156.0    85.0    140     90  well_above_normal  normal   
2   18857  women   165.0    64.0    130     70                NaN  normal   
3   17623    men   169.0    82.0    150    100             normal  normal   
4   17474  women   156.0    56.0    100     60             normal  normal   

    smoke  alco  active  cardio  
id                               
0       0     0       1       0  
1       0     0       1       1  
2       0     0       0       1  
3       0     0       1       1  
4       0     0       0       0

## Initializations 

Declerations of all column types and target column

In [7]:
category_features = [["cholesterol"], ["gluc"]]
binary_features = [["gender"], ["smoke"], ["alco"], ["active"]]
numeric_features = [["age"], ["height"], ["weight"], ["ap_hi"], ["ap_lo"]]
target = "cardio"

Split the data into features and labels

In [14]:
X = df_data.copy()
y = X.pop(target)

## Split data to train test datasets

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

## Custom Transformers
### We create custom transformers for feature engineering

#### Blood Pressure Transformer

Custom transformer responsible for the creation of a new blood pressure categorical feature based on systolic (the number at the top) and diastolic (the number at the bottom) values.  
The transformer will create a new categorical feature with values according to the American Heart Association ranges of blood pressure:
* normal  
* elevated  
* high_pressure_stage_1  
* high_pressure_stage_2  
* hypertensive_crisis  
  
  
![title](https://www.inquirer.com/resizer/jACzP-uAGFwY4JBAfo7tY6qUj-k=/1400x932/smart/arc-anglerfish-arc2-prod-pmn.s3.amazonaws.com/public/PHXNJPTEGRB4RCDYCFJZ6IBOZY.jpg)  
Photo from [American Heart Association](https://www.heart.org/-/media/data-import/downloadables/pe-abh-what-is-high-blood-pressure-ucm_300310.pdf?la=en&hash=CAC0F1D377BDB7BC3870993918226869524AAC3D)

In [38]:
class BloodPressureTransformer(BaseEstimator, TransformerMixin):
    
    def __init__(self):
        
        # Systolic and diastolic blood pressure ranges based on the American Heart Association
        self.systolic_ranges = [-np.inf, 119, 129, 139, 180, np.inf]
        self.diastolic_ranges = [-np.inf, 79, 89, 120, np.inf]
        
        # Blood pressure categories
        self.blood_pressure_category = ["normal", "elevated", "high_pressure_stage_1", "high_pressure_stage_2", "hypertensive_crisis"]
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        # Copy the data so we will not change the original instance
        df_blood_pressure = X.copy()
        
        # Break down ranges of systolic values to categories
        df_blood_pressure["systolic"] = pd.cut(df_blood_pressure["ap_hi"], self.systolic_ranges, labels=["<120", "120-129", "130-139", "140-180", ">180"])
        
        # Break down ranges of diastolic values to categories
        df_blood_pressure["diastolic"] = pd.cut(df_blood_pressure["ap_lo"], self.diastolic_ranges, labels=["<79", "80-89", "90-120", ">120"])
        
        # Combine ranges from systolic and diastolic features to determine the category of the blood pressure feature
        df_blood_pressure.loc[(df_blood_pressure["systolic"] == "<120") &
                              (df_blood_pressure["diastolic"] == "<79"), "blood_pressure"] = self.blood_pressure_category[0]
        
        df_blood_pressure.loc[(df_blood_pressure["systolic"] == "120-129") &
                              (df_blood_pressure["diastolic"] == "<79"), "blood_pressure"] = self.blood_pressure_category[1]
        
        df_blood_pressure.loc[(df_blood_pressure["systolic"] == "130-139") |
                              (df_blood_pressure["diastolic"] == "80-89"), "blood_pressure"] = self.blood_pressure_category[2]
        
        df_blood_pressure.loc[(df_blood_pressure["systolic"] == "140-180") |
                              (df_blood_pressure["diastolic"] == "90-120"), "blood_pressure"] = self.blood_pressure_category[3]
        
        df_blood_pressure.loc[(df_blood_pressure["systolic"] == ">180") |
                              (df_blood_pressure["diastolic"] == ">120"), "blood_pressure"] = self.blood_pressure_category[4]
        
        # Return blood pressure feature as a dataframe with one column
        return df_blood_pressure[["blood_pressure"]]

### Unhealty Lifestyle Transformer

Custom transformer responsible for the creation of a new "unhealty lifestyle" feature.  
This is a boolean feature representing the use of cigarettes, alcohol, and physical inactivity. 

In [36]:
class UnhealtyLifestyleTransformer(BaseEstimator, TransformerMixin):
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        # Copy the data so we will not change the original instance
        df_unhealty_lifestyle = X.copy()
        
        # If you smoke or use alcohol or don't do physical activity, you maintain an unhealty lifestyle!
        df_unhealty_lifestyle["unhealty_lifestyle"] = df_unhealty_lifestyle["smoke"] | df_unhealty_lifestyle["alco"] | (1 - df_unhealty_lifestyle["active"])
        
        # Return unhealty lifestyle feature as a dataframe with one column
        return df_unhealty_lifestyle[["unhealty_lifestyle"]]

## Definition of DataFrameMapper transformers

Now we will define the pipeline of transformations and the raw features we need to complete the creation and processing of the new features and the original features.  
We will pass this to the DataFrameMapper class of the sklearn-pandas package.

In [35]:
# Input features "ap_hi", "ap_lo".
# Steps:
#    BloodPressureTransformer - create blood pressure feature based on "ap_hi", "ap_lo".
#    SimpleImputer - fill nans with the most frequent value.
#    OneHotEncoder - encode categorical values as a one-hot numeric array.
gen_blood_pressure = (
    ["ap_hi", "ap_lo"],
    [
        BloodPressureTransformer(),
        SimpleImputer(strategy="most_frequent"),
        OneHotEncoder()
    ],
    {"alias": "blood_pressure"}
)

gen_blood_pressure

(['ap_hi', 'ap_lo'],
 [BloodPressureTransformer(),
  SimpleImputer(strategy='most_frequent'),
  OneHotEncoder()],
 {'alias': 'blood_pressure'})

In [39]:
# Input features ["smoke", "alco", "active"].
# Steps:
#    UnhealtyLifestyleTransformer - create unhealty lifestyle feature based on "smoke", "alco", "active".
#    SimpleImputer - fill nans with the most frequent value.
gen_unhealty_lifestyle = (
    ["smoke", "alco", "active"],
    [
        UnhealtyLifestyleTransformer(),
        SimpleImputer(strategy="most_frequent")
    ],
    {"alias": "unhealty_lifestyle"}
)

gen_unhealty_lifestyle

(['smoke', 'alco', 'active'],
 [UnhealtyLifestyleTransformer(), SimpleImputer(strategy='most_frequent')],
 {'alias': 'unhealty_lifestyle'})

### Apply the same transformers for multiple columns with gen_features

In [40]:
# Input features [["cholesterol"], ["gluc"]] (The columns are now list of lists because we want to send 2-dimentional DataFrame to each of the transformers).
# Steps:
#    SimpleImputer - fill nans with the most frequent value.
#    OneHotEncoder - encode categorical values as a one-hot numeric array.
gen_category = gen_features(
    columns=category_features,
    classes=[
        {
            "class": SimpleImputer,
            "strategy": "most_frequent"
        },
        {
            "class": OneHotEncoder
        }
    ]
)

gen_category

[(['cholesterol'],
  [SimpleImputer(strategy='most_frequent'), OneHotEncoder()],
  {}),
 (['gluc'], [SimpleImputer(strategy='most_frequent'), OneHotEncoder()], {})]

In [41]:
# Input features [["gender"], ["smoke"], ["alco"], ["active"]] (The columns are now list of lists because we want to send 2-dimentional DataFrame to each of the transformers).
# Steps:
#    SimpleImputer - fill nans with the most frequent value.
#    OrdinalEncoder - encode categorical features as an integer array.
gen_binary = gen_features(
    columns=binary_features,
    classes=[
        {
            "class": SimpleImputer,
            "strategy": "most_frequent"
        },
        {
            "class": OrdinalEncoder
        }
    ]
)

gen_binary

[(['gender'], [SimpleImputer(strategy='most_frequent'), OrdinalEncoder()], {}),
 (['smoke'], [SimpleImputer(strategy='most_frequent'), OrdinalEncoder()], {}),
 (['alco'], [SimpleImputer(strategy='most_frequent'), OrdinalEncoder()], {}),
 (['active'], [SimpleImputer(strategy='most_frequent'), OrdinalEncoder()], {})]

In [42]:
# Input features [["age"], ["height"], ["weight"], ["ap_hi"], ["ap_lo"]] (The columns are now list of lists because we want to send 2-dimentional DataFrame to each of the transformers).
# Steps:
#    SimpleImputer - fill nans with the mean value.
#    StandardScaler - standardize features by removing the mean and scaling to unit variance.
gen_numeric = gen_features(
    columns=numeric_features,
    classes=[
        {
            "class": SimpleImputer,
            "strategy": "mean"
        },
        {
            "class": StandardScaler
        }
    ]
)

gen_numeric

[(['age'], [SimpleImputer(), StandardScaler()], {}),
 (['height'], [SimpleImputer(), StandardScaler()], {}),
 (['weight'], [SimpleImputer(), StandardScaler()], {}),
 (['ap_hi'], [SimpleImputer(), StandardScaler()], {}),
 (['ap_lo'], [SimpleImputer(), StandardScaler()], {})]

### DataFrameMapper Construction

Now we will define the course of action of the DataFrameMapper and indicate that the input and output will be Pandas Dataframe.

In [43]:
preprocess_mapper = DataFrameMapper(
    [
        gen_blood_pressure,
        gen_unhealty_lifestyle,
        *gen_category,
        *gen_binary,
        *gen_numeric,
    ],
    input_df=True,
    df_out=True
)

In [44]:
preprocess_mapper.fit_transform(X_train, y_train).head()

blood_pressure_x0_elevated  blood_pressure_x0_high_pressure_stage_1  \
id                                                                           
98125                         0.0                                      0.0   
28510                         0.0                                      1.0   
15795                         0.0                                      0.0   
39560                         0.0                                      0.0   
32677                         0.0                                      0.0   

       blood_pressure_x0_high_pressure_stage_2  \
id                                               
98125                                      1.0   
28510                                      0.0   
15795                                      1.0   
39560                                      1.0   
32677                                      1.0   

       blood_pressure_x0_hypertensive_crisis  blood_pressure_x0_normal  \
id                                                                       
98125                                    0.0                       0.0   
28510                                    0.0                       0.0   
15795                                    0.0                       0.0   
39560                                    0.0                       0.0   
32677                                    0.0                       0.0   

       unhealty_lifestyle  cholesterol_x0_above_normal  cholesterol_x0_normal  \
id                                                                              
98125                   0                          0.0                    0.0   
28510                   1                          0.0                    0.0   
15795                   0                          0.0                    1.0   
39560                   0                          0.0                    0.0   
32677                   0                          0.0                    0.0   

       cholesterol_x0_well_above_normal  gluc_x0_above_normal  ...  \
id                                                             ...   
98125                               1.0                   0.0  ...   
28510                               1.0                   0.0  ...   
15795                               0.0                   0.0  ...   
39560                               1.0                   0.0  ...   
32677                               1.0                   0.0  ...   

       gluc_x0_well_above_normal  gender  smoke  alco  active       age  \
id                                                                        
98125                        0.0     1.0    0.0   0.0     1.0  0.388175   
28510                        1.0     0.0    0.0   0.0     0.0  1.308468   
15795                        0.0     1.0    0.0   0.0     1.0  1.346527   
39560                        0.0     1.0    0.0   0.0     1.0  1.291463   
32677                        0.0     1.0    0.0   0.0     1.0  0.912495   

         height        weight     ap_hi     ap_lo  
id                                                 
98125 -0.549199  1.024035e-15 -0.056261 -0.035475  
28510  0.333478 -6.621198e-01 -0.056261 -0.085423  
15795 -0.549199 -5.900598e-01 -0.056261 -0.035475  
39560 -0.170909 -1.382720e+00 -0.024632 -0.035475  
32677 -0.801392  7.790809e-01  0.133513 -0.085423  

[5 rows x 21 columns]

### Feature Selection

In the feature selection step, we specify that the input columns are the transform columns of the previous preprocessing step and then specify the feature selection transformer we want to use.  
In our case I choose SelectFromModel with RandomForestClassifier to select the most important features based on feature_importances_ attribute of RandomForestClassifier. 

In [45]:
preprocess_mapper.transformed_names_

['blood_pressure_x0_elevated',
 'blood_pressure_x0_high_pressure_stage_1',
 'blood_pressure_x0_high_pressure_stage_2',
 'blood_pressure_x0_hypertensive_crisis',
 'blood_pressure_x0_normal',
 'unhealty_lifestyle',
 'cholesterol_x0_above_normal',
 'cholesterol_x0_normal',
 'cholesterol_x0_well_above_normal',
 'gluc_x0_above_normal',
 'gluc_x0_normal',
 'gluc_x0_well_above_normal',
 'gender',
 'smoke',
 'alco',
 'active',
 'age',
 'height',
 'weight',
 'ap_hi',
 'ap_lo']

In [46]:
feature_selection = DataFrameMapper(
    [(
        preprocess_mapper.transformed_names_,
        SelectFromModel(RandomForestClassifier(n_estimators=100))
    )]
)

### Sklearn Pipeline

Now we piece together all prevoius definitions to define the full pipeline:
* preprocessing
* feature selection
* estimator

In [47]:
pipeline = Pipeline(steps=[
    ("preprocess", preprocess_mapper),
    ("feature_selection", feature_selection),
    ("estimator", RandomForestClassifier(n_estimators=100, max_depth=6))
])

In [48]:
pipeline.fit(X_train, y_train)
preds = pipeline.predict(X_test)
preds

array([1, 1, 1, ..., 1, 1, 1], dtype=int64)

## Grid Search

Next I created a grid search object which includes the original pipeline.  
When I then call fit, the transformations are applied to the data, before a cross-validated grid-search is performed over the parameter grid.

In [31]:
pipeline = Pipeline(steps=[
    ("preprocess", preprocess_mapper),
    ("feature_selection", feature_selection),
    ("estimator", RandomForestClassifier())
])

In [32]:
param_grid = { 
    "estimator__n_estimators": [200, 500],
    "estimator__max_features": ['auto', 'sqrt', 'log2'],
    "estimator__max_depth": [4, 5, 6, 7, 8],
    "estimator__criterion":['gini', 'entropy']
}

In [33]:
gscv_estimator = GridSearchCV(pipeline, param_grid, cv=5, n_jobs=-1)
gscv_estimator.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocess',
                                        DataFrameMapper(df_out=True,
                                                        drop_cols=[],
                                                        features=[(['ap_hi',
                                                                    'ap_lo'],
                                                                   [BloodPressureTransformer(),
                                                                    SimpleImputer(strategy='most_frequent'),
                                                                    OneHotEncoder()],
                                                                   {'alias': 'blood_pressure'}),
                                                                  (['smoke',
                                                                    'alco',
                                                                    'active'],
             

In [49]:
display(gscv_estimator.best_params_)

{'estimator__criterion': 'gini',
 'estimator__max_depth': 8,
 'estimator__max_features': 'auto',
 'estimator__n_estimators': 500}

In [56]:
preds = gscv_estimator.predict(X_test)
np.unique(preds,return_counts=True)

(array([0, 1], dtype=int64), array([11408,  9592], dtype=int64))